In [5]:
import pandas as pd
import os
import sys
from pathlib import Path
from prfndim.prfndim import Z, P, C, R, S
import numpy as np

In [6]:
path = Path.cwd().parent / "data" / "prfndim" / "raw_prfndim.csv"
df = pd.read_csv(path)

arities = []
for index, row in df.iterrows():
    arities.append(eval(row["expr"]).arity)
df["arity"] = arities
print(df)

                                        expr  arity
0                                        Z()    NaN
1                                        S()    1.0
2                                    P(1, 1)    1.0
3                                    P(2, 1)    2.0
4                                    P(2, 2)    2.0
...                                      ...    ...
9976                         C(S(), P(2, 2))    2.0
9977  R(P(2, 2), P(3, 1), P(3, 1), Z(), Z())    1.0
9978   R(S(), C(P(2, 1), Z(), P(2, 2)), Z())    1.0
9979  R(P(2, 1), P(3, 2), P(3, 3), Z(), Z())    1.0
9980  R(P(2, 1), P(3, 3), P(3, 1), Z(), Z())    1.0

[9981 rows x 2 columns]


In [ ]:
df_unique = df.drop_duplicates(subset=["expr"])
print(df_unique)

                                                   expr  arity
0                                                   Z()    NaN
1                                                   S()    1.0
2                                               P(1, 1)    1.0
3                                               P(2, 1)    2.0
4                                               P(2, 2)    2.0
...                                                 ...    ...
9960  C(R(P(3, 1), P(4, 2), P(4, 2), P(4, 1), Z(), Z...    3.0
9963         C(P(3, 2), R(S(), P(2, 2), Z()), Z(), Z())    1.0
9964  C(C(P(3, 1), C(P(1, 1), Z()), C(P(1, 1), Z()),...    3.0
9973  C(R(S(), C(P(2, 1), Z(), P(2, 2)), Z()), P(4, 4))    4.0
9975  C(C(P(3, 3), P(3, 3), P(3, 3), Z()), C(R(S(), ...    4.0

[4230 rows x 2 columns]


In [11]:
sample_max = 10
sample_num = 5
max_p_arity = 4

eq_domain = [np.zeros((1))] + [
    np.random.randint(1, sample_max + 1, size=(sample_num, dim))
    for dim in range(1, max_p_arity + 1)
]
eq_domain[1] = np.array(range(1, sample_max + 1)).reshape(-1, 1)

outputs_li = []
inputs_li = []
len_li= []
for index, row in df_unique.iterrows():
    expr = eval(row["expr"])
    if expr.arity is None:
        inputs = np.full((sample_num, 1), 0)
    else:
        inputs = eq_domain[expr.arity]
    outputs = tuple(expr.eval(*args) for args in inputs)
    inputs_li.append(tuple(tuple(int(arg) for arg in args) for args in inputs))
    outputs_li.append(tuple(int(output) for output in outputs))
    len_li.append(len(str(expr)))
df_unique["inputs"] = inputs_li
df_unique["outputs"] = outputs_li
df_unique["len"] = len_li


df_unique_output = df_unique.drop_duplicates(subset=["arity", "outputs"])
df_unique_output = df_unique_output.sort_values(by=["arity", "len"]).reset_index(drop=True)
print(df_unique_output)

                                                  expr  arity  \
0                                                  S()    1.0   
1                                              P(1, 1)    1.0   
2                                          C(S(), S())    1.0   
3                          C(C(S(), S()), C(S(), S()))    1.0   
4        R(S(), C(R(S(), P(2, 2), Z()), P(2, 2)), Z())    1.0   
..                                                 ...    ...   
179  C(R(P(3, 3), C(R(P(2, 2), P(3, 1), P(3, 2), Z(...    4.0   
180  C(C(R(P(2, 2), P(3, 1), P(3, 1), Z(), Z()), P(...    4.0   
181                                                Z()    NaN   
182  C(P(2, 2), C(S(), Z()), C(S(), C(P(1, 1), C(S(...    NaN   
183  C(R(S(), C(P(2, 1), Z(), P(2, 2)), C(P(1, 1), ...    NaN   

                                                inputs  \
0    ((1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,...   
1    ((1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,...   
2    ((1,), (2,), (3,), (4,), (5,), (6,), (7,

/tmp/ipykernel_3648/1165187328.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["inputs"] = inputs_li
/tmp/ipykernel_3648/1165187328.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["outputs"] = outputs_li
/tmp/ipykernel_3648/1165187328.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [12]:
df_unique_output.to_csv("../data/prfndim/unique_output.csv", index=True)